# **Modelos para detección de transacciones financieras fraudulentas**

## **Descripción**

En este notebook se entrenan y evalúan varios modelos de Machine Learning (Logistic Regression, Random Forest y XGBoost) para la detección de transacciones fraudulentas en tarjetas de crédito. El objetivo es comparar distintas técnicas de clasificación y escoger la mejor en términos de recall, precision y otras métricas adecuadas para un problema altamente desbalanceado.  

Los apartados tratados son:
- Carga del dataset tras un EDA previo.
- División en entrenamiento y test (estratificado).
- Entrenamiento de varios algoritmos.
- Evaluación con métricas (matriz de confusión, AUPRC, etc.).
- Comparación de los resultados.
- Conclusiones finales.

## **Módulos**

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    precision_recall_curve, auc, roc_auc_score
)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

## **Cargo dataset tras EDA**

In [73]:
df = pd.read_csv("creditcard_clean.csv") 
print("Tamaño del dataset:", df.shape)

Tamaño del dataset: (283726, 31)


## **Preparación de features y target**

In [74]:
X = df.drop(columns=['class'])  # El resto de columnas son predictoras
y = df['class']                 # Clase (0: No Fraude, 1: Fraude)

## **División en entrenamiento y test (estratificado por estar desbalanceado)**

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=7
)

print("\nDistribución de clases en entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de clases en test:")
print(y_test.value_counts(normalize=True))


Distribución de clases en entrenamiento:
class
0    0.998332
1    0.001668
Name: proportion, dtype: float64

Distribución de clases en test:
class
0    0.998336
1    0.001664
Name: proportion, dtype: float64


## **Función para evaluar los modelos**

In [ ]:
def evaluar_modelo(model, X_test, y_test, umbral=0.5):
    """
    Imprime matriz de confusión, metrics, AUPRC, ROC-AUC, etc.
    umbral es el corte para predecir 0 o 1 a partir de la probabilidad
    """
    
    y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva (fraude=1)
    y_pred = (y_proba >= umbral).astype(int)     # Predicciones binarias, según umbral
    
    cm = confusion_matrix(y_test, y_pred)        # Matriz de confusión
    
    # Precision, Recall, AUPRC
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)
    
    roc_auc = roc_auc_score(y_test, y_proba)     # ROC AUC
    
    # Clasification Report
    clf_report = classification_report(y_test, y_pred, digits=4)
    
    print("Matriz de Confusión:")
    print(cm)
    print("\nReporte de Clasificación:")
    print(clf_report)
    print(f"AUPRC = {pr_auc:.4f}")
    print(f"ROC-AUC = {roc_auc:.4f}")

## **Entrenamiento de los modelos**

### **Regresión logística**

In [ ]:
print("============================================")
print("     MODELO 1: LOGISTIC REGRESSION")
print("============================================")
lr = LogisticRegression(
    solver='lbfgs',
    max_iter=10000,
    random_state=7
)

lr.fit(X_train, y_train)
evaluar_modelo(lr, X_test, y_test, umbral=0.3)